In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"
cohort = "GSE65161"

# Input paths
in_trait_dir = "../../input/GEO/Essential_Thrombocythemia"
in_cohort_dir = "../../input/GEO/Essential_Thrombocythemia/GSE65161"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/GSE65161.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE65161.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE65161.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Mediator kinase inhibition further activates super-enhancer-associated genes in AML"
!Series_summary	"Super-enhancers (SEs), which are composed of large clusters of enhancers densely loaded with the Mediator complex, transcription factors and chromatin regulators, drive high expression of genes implicated in cell identity and disease, such as lineage-controlling transcription factors and oncogenes. BRD4 and CDK7 are positive regulators of SE-mediated transcription. By contrast, negative regulators of SE-associated genes have not been well described. Here we show that the Mediator-associated kinases cyclin-dependent kinase 8 (CDK8) and CDK19 restrain increased activation of key SE-associated genes in acute myeloid leukaemia (AML) cells. We report that the natural product cortistatin A (CA) selectively inhibits Mediator kinases, has anti-leukaemic activity in vitro and in vivo, and disproportionately induces upregulation of SE-associated genes in CA

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Optional, Callable, Dict, Any

def convert_trait(value):
    if value is None:
        return None
    
    value_str = str(value)
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    # Convert cell type to binary for Essential Thrombocythemia
    if 'AML' in value_str:
        return 1  # Has the disease - Acute Myeloid Leukemia
    elif 'CML' in value_str:
        return 0  # Different disease - Chronic Myelogenous Leukemia
    else:
        return None  # Unknown or other conditions

def get_feature_data(clinical_df, row_idx, feature_name, convert_func):
    """Helper function to extract feature data from clinical DataFrame."""
    # Extract row
    row_data = clinical_df.iloc[row_idx]
    
    # Create a new Series for the feature
    feature_data = pd.Series(
        [convert_func(val) for val in row_data.values],
        index=row_data.index,
        name=feature_name
    )
    
    return pd.DataFrame(feature_data)

# Trait data is available in row 2 (index) where cell type is mentioned
trait_row = 2

# No age data available in the sample characteristics
age_row = None
convert_age = None

# No gender data available in the sample characteristics
gender_row = None
convert_gender = None

# Gene expression data availability
# Based on the background information, this is likely genomics data (not only miRNA or methylation)
is_gene_available = True

# Trait data availability
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical feature extraction if trait data is available
if trait_row is not None:
    # Assuming clinical_data was previously loaded
    clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_file):
        clinical_data = pd.read_csv(clinical_data_file, index_col=0)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:", preview)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to a CSV file
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The gene identifiers shown (like '1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that store probe IDs and gene symbols
# From the preview, we can see:
# - The 'ID' column contains probe IDs like '1007_s_at' which match the gene expression data
# - The 'Gene Symbol' column contains gene symbols like 'DDR1 /// MIR4640'

# 2. Get a gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Check the mapping dataframe
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the transformed gene expression data
print("After mapping to gene symbols - first 10 genes:")
print(gene_data.index[:10])
print("Gene expression data shape:", gene_data.shape)

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print("After normalizing gene symbols - shape:", gene_data.shape)

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
After mapping to gene symbols - first 10 genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')
Gene expression data shape: (21278, 24)


After normalizing gene symbols - shape: (19845, 24)


Gene expression data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/GSE65161.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - this was already done in Step 6
# We'll reuse the normalized gene_data from the previous step

# 2. Link the clinical and genetic data
# First, we need to properly extract the clinical features as identified in Step 2
print("Extracting clinical features...")
# Create the clinical features using the trait row identified in Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from leukemia cell lines including Essential Thrombocythemia relevant AML subtypes."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Extracting clinical features...
Clinical data preview:
{'GSM1585803': [0.0], 'GSM1585804': [0.0], 'GSM1585805': [0.0], 'GSM1585806': [0.0], 'GSM1585807': [0.0], 'GSM1585808': [0.0], 'GSM1585875': [1.0], 'GSM1585876': [1.0], 'GSM1585877': [1.0], 'GSM1585878': [1.0], 'GSM1585879': [1.0], 'GSM1585880': [1.0], 'GSM1585881': [1.0], 'GSM1585882': [1.0], 'GSM1585883': [1.0], 'GSM1585884': [1.0], 'GSM1585885': [1.0], 'GSM1585886': [1.0], 'GSM1585924': [1.0], 'GSM1585925': [1.0], 'GSM1585926': [1.0], 'GSM1585927': [1.0], 'GSM1585928': [1.0], 'GSM1585929': [1.0]}
Clinical data saved to ../../output/preprocess/Essential_Thrombocythemia/clinical_data/GSE65161.csv
Linking clinical and genetic data...
Linked data shape: (24, 19846)
Handling missing values...


Linked data shape after handling missing values: (24, 19846)
Checking for bias in trait distribution...
For the feature 'Essential_Thrombocythemia', the least common label is '0.0' with 6 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Essential_Thrombocythemia/GSE65161.csv
